Qeusta prima parte serve ad estrarre una prima vaersione grezza dei dati dal pdf

In [1]:
import json
def coordinate():
    nomiT = []    # dichiaro l'array links
    dati=[]
    
    with open('coordinate-1.json') as json_file:
        datiCoordinate = json.load(json_file)
        #print(datiCoordinate)
        dati = datiCoordinate
        print("***************************")

        for nome in datiCoordinate:
            #print(nome)
        
            nomiT.append(nome)    # metto i links recuperati da "dataLinks", in "links"  
    #print(dati[nome][1])
    
    return(nomiT, dati)

In [2]:
#coordinate() # dichiaro la mia funzione che mi restituisce le coordinate delle tavole con la pagina, da passare per l'estrazione
#var = coordinate()
#nomi= var[0]
#cont= var[1]
#i = 1 #un ciclo da 0 a len var[0] | var[1] anche se dovrebbero essere uguali
#var[1][nomi[i]][0] #mi seleziona le coordinate 
#var[1][nomi[i]][1] #mi seleziona la pagina
#var[0][i] #mi seleziona il nome tavola
#var[0][4]

sopra andrò ad importare il mio json con le coordinate

Il tutto funziona, va ripulito scegliendo cosa mi serve e messo in un ciclo che mi guarda tutte le tavole che voglio

In [3]:
#da saltare

#var = coordinate()
#for i in range(0, len(var[0])):
#    print(var[1][var[0][i]][1])
    

In [4]:
from tabula import read_pdf

var = coordinate()
df=[]

for i in range(0, len(var[0])):
    
    # [top,left,bottom,width]
    box = var[1][var[0][i]][0]
    pagina = var[1][var[0][i]][1]

    # fattore di conversione
    fc = 28.28
         
    for j in range(0, len(box)):
        box[j] /= 10
        box[j] *= fc
    
    df.append(read_pdf("dati/Utilizzo-e-qualità-della-risorsa-idrica-in-Italia.pdf", pages=pagina,area=[box],output_format="json", stream=True))

#df    #contiene tutte le tabelle

#questo me lo stampa in una maniera più facilmente leggibile ad un umano

***************************


In [ ]:
#df

La prima e la seconda riga per la mia prima tabellla almeno, corrispondono al titolo della tabella

il codice commentato sotto, serve a visualizzare le singole righe con il rispettivo contenuto in testo

In [ ]:
# prima colonna
#df[0][0]['data'][0][0]['text'] 
# seconda colonna
#df[0]['data'][1][0]['text']


Crea la tabella con i miei dati

In [ ]:
import re 

def tab_fin():
    tab=[]
#test=[]    --> INUTILE
    fin=[]

    nome_tab = re.findall("[A-Za-z]+\s[0-9]+\.[0-9]+", table[0])  #contiene il nome tabella sul pdf es. Tavola 1.1
    fonte_tab = table[len(df[1][0]['data'])-1]    #contiene la fonte della tab (es istat)

    for i in range(1,len(table)):    #cicla partendo da uno, in modo da evitare il nome della tabella
        x = re.findall("([A-Za-z]+\s+)([0-9]+|.,[0-9]+)", table[i])   #mi seleziona tutti i nomi (di regione) seguiti da numeri  
    # forse la riga sopra va raffinata xke potrebbe dare problemi in tab he hanno nomi seguiti da numeri in righe che non sono la prima ma nemmeno le regioni
    #ATT --> inoltre la RE --> ([A-Za-z]\s+)([0-9]) presente sotto in y, dovrebbe fare la stessa cosa, alivello di utilità qui,
    #quindi potrei acnhe sostituire questa con quella RE
        if x:
            tab.append(table[i])    #ora tab contien tutti i dati necessari estratti dal pdf, sono da riordinare

    for j in range(0,len(tab)):
    #([A-Za-z]\s+)([0-9])    --> INUTILE
    #Y=re.findall("([A-Za-z]\s+)([0-9])", tab[j])    --> INUTILE
        y=re.search("([A-Za-z]\s+)([0-9])", tab[j])   #mi trova tutti i nomi delle regioni + gli spazi + il primo numero
        if y:    # toglie il priomo numero dal nome delle regioni ma lascia gli spazi
            val=y.span()   #trova l'estensione della parola
            nome=tab[j][0:val[1]-1]   #contiene il nome della regione (della riga)
            nr=tab[j].split(nome)     #contiene il valore della riga (+ uno spazio bianco iniziale che sotto viene omesso)
        #test.append(nr[1])    --> INUTILE
        #*********************************
            riga_fin = str(nome)+":"+str(nr[1])
        #fin.append(z)   #aggiunge il nome della regione    --> INUTILE
        #fin.append(num)   #aggiunge i valori               --> INUTILE
            fin.append(riga_fin)

#fin.append(fonte_tab)    aggiunge la fonte alla tab, vedere come gestire la cosa

    print("*************************")
#test                             --> INUTILE
#ok    test.append(z)             --> INUTILE
        #test.append(y.span())    --> INUTILE
    return fin
#test    --> INUTILE
#tab   # già il risultato di tab lo posso dare in pasto ad openrefine e suddividere in colonne da li     --> INUTILE

In [5]:
def export(x,y):
    #x = nome_tab[0]
    #y = fin
    import json
    import os
    #if x == 0 or "" or None:
        #x = mancante
        #z = "questa tabella ha dei problemi di esportazione"
        #with open("tab/"+x+".json", "w") as outfile:
            #json.dump(z, outfile, indent=3)
        #print("tab problematica")
    #else:
    with open("tab/"+x[0]+".json", "w") as outfile:
        json.dump(y, outfile, indent=3)
    
# il nome della tabella corrispode a quello sul pdf (Tavola *.*)
# esporto tutto su openrefine e divido i nomi dai valori e poi uso la RE di seguito --> (["   "]+|["  "]+|[" "]+)
# il risultato è la mia tabella corretta

******************************************************************************************************************

In [6]:
import re 

def tabella_grezza():
    for j in range(0,len(df)):
        table = []
        for i in range(0,len(df[j][0]['data'])):
            if df[j][0]['data']:
                #for j in range(0, len(df[0][0]['data'][i][0]['text'])):
                    riga = df[j][0]['data'][i][0]['text']
                    table.append(riga)
#qui devo inserire la parte tab_fin che crea la tabella definitiva 
#sotto va la parte che la esporta in un file 
#***************************************************************************************************************
        tab=[]
        fin=[]

        nome_tab = re.findall("[A-Za-z]+\s[0-9]+\.[0-9]+", table[0])  #contiene il nome tabella sul pdf es. Tavola 1.1
        fonte_tab = table[len(df[j][0]['data'])-1]    #contiene la fonte della tab (es istat)

        for i in range(1,len(table)):    #cicla partendo da uno, in modo da evitare il nome della tabella
            x = re.findall("([A-Za-z]+\s+)([0-9]+|.,[0-9]+)", table[i])   #mi seleziona tutti i nomi (di regione) seguiti da numeri  
            if x:
                tab.append(table[i])    #ora tab contien tutti i dati necessari estratti dal pdf, sono da riordinare

        for j in range(0,len(tab)):
            y=re.search("([A-Za-z]\s+)([0-9])", tab[j])   #mi trova tutti i nomi delle regioni + gli spazi + il primo numero
            if y:    # toglie il priomo numero dal nome delle regioni ma lascia gli spazi
                val=y.span()   #trova l'estensione della parola
                nome=tab[j][0:val[1]-1]   #contiene il nome della regione (della riga)
                nr=tab[j].split(nome)     #contiene il valore della riga (+ uno spazio bianco iniziale che sotto viene omesso)
        #*********************************
                riga_fin = str(nome)+":"+str(nr[1])
                fin.append(riga_fin)
        print("*************************")
        print(nome_tab)
#***************************************************************************************************************
        export(nome_tab,fin)
    return table

In [7]:
tabella_grezza()

*************************
['Tavola 1.1']
*************************
['Tavola 1.2']
*************************
['Tavola 1.5']
*************************
['Tavola 1.6']
*************************
['Tavola 1.8']
*************************
['Tavola 1.9']
*************************
['Tavola 1.10']
*************************
['Tavola 2.2']
*************************
['Tavola 3.2']


['Tavola 3.2 – Goal 6: indicatori diffusi da Istat in serie storica',
 'Indicatori',
 '6.1.1 6.1.1 6.1.1 6.3.1',
 'ANNI Famiglie che Famiglie che Acqua erogata Quota percentuale dei carichi',
 'non si fidano di lamentano pro capite inquinanti confluiti in impianti',
 'bere l’acqua del irregolarità (litri/ab/g) secondari o avanzati rispetto',
 'rubinetto (%) nell’erogazione ai carichi complessivi urbani',
 'di acqua (%) generati (%)',
 '2005 - - 250 53,5',
 '2008 - - 253 56,5',
 '2010 32,8 10,8 - -',
 '2011 30,0 9,4 - -',
 '2012 30,3 8,9 241 57,6',
 '2013 29,2 10,0 - -',
 '2014 28,0 8,7 - -',
 '2015 30,0 9,2 220 59,6',
 '2016 29,9 9,4 - -',
 '2017 29,1 10,1 - -',
 '2018 29,0 10,4 - -']

Non ha molto senso dividere il numero dalla regione qui visto che posso usare open refine 
inoltre per poterla fare qui, dovrei fare una RE specifica che mi riconosca i numeri fatti in una certa maniera

a gruppi di tre, divisi da un punto, e l'ultimo gruppo diviso da una virgola, molto più facile far fare ad open refine 



In [ ]:
table[len(df[0]['data'])-1]

In [ ]:
#ATT  non ha molto senso fare la divisione dei numeri qui xke posso usare openrefine
# ATT PROBABILMENTE INUTILE DOVRO' CANCELLARE TUTTA LA CELLA

import re 

tab=[]
temp=[]

for i in range(1,len(table)):
    x = re.findall("([A-Za-z]+\s+)([0-9]+|.,[0-9]+)", table[i])
    if x:
        temp.append(table[i])
        
for j in range(0,len(temp)):
    y = re.split("([0-9]+|.,[0-9]+\s*)|(\s-\s)", temp[j])
    tab.append(y)

        
print("************************")
print(tab)
print("************************")

*************************************************************************************

In [ ]:
# elimino le colonne vuote
table = []
for i in range(0,len(df)):
    riga = {}
    riga['regione'] = regioni[i]
    if df[i]['data']:
        for j in range(0,len(df[i]['data'])):
            primaColonna = df[i]['data'][j][0]['text']
            if primaColonna != '' and primaColonna != '17 operatori sanitari' and primaColonna != "Fascia d'età":
                if primaColonna == '&gt;90':
                    primaColonna = '> 90'
                secondaColonna = df[i]['data'][j][1]['text']
                riga[primaColonna] = secondaColonna
    table.append(riga)
table 

In [ ]:
# salvo i risultati su file 
import csv
nomi_colonne = table[0].keys()
with open('dati-regioni-covid.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, nomi_colonne)
    dict_writer.writeheader()
    dict_writer.writerows(table)